## Chapter 9 - Clustering Methods

Clustering refers to a broad set of techniques finding subgroups, or clusters in a dataset. When we cluster observations from a dataset, we partition them to distinct groups so the observations within each group are quite similar to each other, while observations across groups are quite different from each other. Consequently, we need to define what is similar or different.

Given $n$ observations, each with $p$ features, we may have a reason to believer there is some heterogeneity among these $n$ observations. Clustering can be used to find these subgroups.

This is an unsupervised learning problem as we attempt to discover structure on the basis of a dataset.

In general, we can cluster either the $n$ observations to identify subgroups among the observations, or the $p$ features to identify subgroups amongst the features. To do the latter, simply transpose the matrix before using any model.

Note that:

- PCA aims to find a low-dimension representation of the observations that explain a good fraction of the variance
- Clustering looks to find homogeneous subgroups amongs the observations.

### k-Means Clustering

$k$-means clustering is a simple and elegant approach to partition a dataset to $k$ distinct, non-overlapping clusters. To perform $k$-means clustering, first specify the desired number of clusters $k$, then the algorithm will assign each observation to one of the $k$ clusters. The assigning of each observation to each cluster $C_1, \cdots, C_k$ such that:
1. $C_1 \cup C_2 \cup C_K = \{i_1,\cdots\ ,i_n\}$. Each observation belongs to at least one of the clusters
2. $C_k \cap C_{k'} = \emptyset \,\,\forall\,\, k \neq k'$. The clusters are non-overlapping

The idea behind $k$-means clustering is that a good clustering is one where the <u>within-cluster variation</u> is as small as possible. The within-cluster variation for a cluster $C_k$, $W(C_k)$ is the amount by which the observations within a cluster differ from each other. 

We hence aim to partition the observations into $K$ clusters such that the total within-cluster variation, summed over $K$ clusters is as small as possible. Mathematically, the optimisation problem is:
$$\underset{C_1, \cdots, C_k}{\text{Minimise }}\left\{\sum_{k=1}^K W(C_k)\right\}$$

The most common choice to define $W$ is the euclidean distance. We attempt to find the sum of all pairwise squared Euclidean distances between the observations in the $k$-th cluster, divided by the total number of observations in the $k$-th cluster.

$$W(C_k) = \frac{1}{|C_k|}\sum_{i\in C_k, i' \in C_k, i\neq i'} \sum_{j=1}^p \begin{pmatrix}x_{ij}-x_{i'j}\end{pmatrix}^2$$
Where $|C_k|$ refers to the number of observations in the $k$-th cluster. 

Combining both equations, we obtain the optimisation problem for $k$-means clustering:

$$\underset{C_1, \cdots, C_k}{\text{Minimise }}\left\{\sum_{k=1}^K \frac{1}{|C_k|}\sum_{i\in C_k, i' \in C_k, i\neq i'} \sum_{j=1}^p \begin{pmatrix}x_{ij}-x_{i'j}\end{pmatrix}^2\right\}$$

The algorithm to solve for this empirically is:
```comments

1. assign a number 1 - K to every observation

2. iterate until cluster assignments stop changing:
    for each cluster 1,...,K:
        A. compute the cluster centroid which is the p-vector that 
        is the means for all observations in the k-th cluster
        
        B. assign each observation to the cluster whose centroid
        is closest
```

When the result no longer changes, a local optimum is reached. Because this algorithm finds a local optimum rather than a global optimum, the results obtained will depend on the initial (random) cluster assignment / initiation in step 1. For this reason, it is important to run the algorithm multiple times from different initialisation points.